In [ ]:
import fitz  
from PIL import Image
import pytesseract
import io
import re
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import easyocr
import numpy as np
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
endpoint = "https://<your-resource-name>.cognitiveservices.azure.com/"
key = "<your-key>"

client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

with open("document.pdf", "rb") as f:
    poller = client.begin_analyze_document("prebuilt-layout", document=f)
    result = poller.result()

for page in result.pages:
    for line in page.lines:
        print(line.content)

In [10]:
from Jiahao.Helper import sort_toc_lines
from Jiahao.Helper import separate_on_number
from Jiahao.Helper import extract_toc_pages
from Jiahao.Helper import is_valid_toc_line


In [28]:
def extract_toc_pages(pdf_path, lang='deu'):
    doc = fitz.open(pdf_path)  # Open PDF file
    toc_pages = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes()))
        text = pytesseract.image_to_string(img, lang=lang)
         
        if "übersicht" in text.lower():
        #if "inhaltsverzeichnis" in text.lower() or "übersicht" in text.lower():
            return text, img 

In [ ]:
def extract_toc_pages(pdf_path, lang='de'):
    doc = fitz.open(pdf_path)
    reader = easyocr.Reader([lang])  # Initialize EasyOCR with German language
    toc_pages = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes()))
        
        # Convert PIL Image to numpy array for EasyOCR
        img_np = np.array(img)
        
        # Run OCR
        result = reader.readtext(img_np, detail=0)  # detail=0 to get only text strings
        
        full_text = "\n".join(result)
        
        if "übersicht" in full_text.lower():
            return full_text

    return None, None  # If no TOC page is found


In [ ]:
text= extract_toc_pages("Resource/Übersicht3.pdf")

print(text)


process_text = separate_on_number(text)
print(process_text)


valid_lines = [line for line in process_text if is_valid_toc_line(line)]
# for i in valid_lines:
#     print(i)

sorted_lines = sort_toc_lines(valid_lines)
# for i in sorted_lines:
#     print(i)



Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\j7843\Documents\DAIA\GEA_llm_pdf_processing_group17\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Technische Angaben
Grass6
KAB
Schraubenverdichteraggregat
Actriocreticn Eqciamcii
FH-SGC
Übersicht
1.
Umgebungsbedingungen
Umgebungstemperatur
max.40 'C
min.5 'C
1. UMGEBUNGSBEDINGUNGEN
2
frei von Flüssigkeitsniederschlägen
2. PROJEKTIERTE
BETRIEBSBEDINGUNGEN
2
2. Projektierte
Betriebsbedingungen
3. HOCHSTZULÄSSIGE
Sättigungstemperatur
~5 *C
+5 'C
BETRIEBSDRÜCKE
Verdichtersaugdruck to
Überhitzung im
10 K
10 K
4. EINSTELLWERTE FÜR
Verdichtersaugstutzen
SICHERHEITSVENTILE
2
Sättigungstemperatur
+27 'C
30 'C
Verdichtungsenddruck tK
5. EINSTELLWERTE DER
Aufladung des Verdichters
nein
nein
SICHERHEITSGERÄTE GEGEN
UNZULÄSSIGE DRÜCKE
3.
Hochstzulässige Betriebsdrücke
6. EINSTELLWERTE DER
SICHERHEITSGERÄTE GEGEN
Kältemittelkreislauf
23 bar Überdruck
UNZULÄSSIGE TEMPERATUREN
3
Ölkreislauf
23 bar Überdruck
7. SONSTIGE EINSTELLWERTE VON
SCHALTERN UND ARMATUREN
4. Einstellwerte für Sicherheitsventile
8. RICHTWERTE BETRIEB DES
Sicherheitsventil (3.1)
23 bar Überdruck
VERDICHTERAGGREGATES
Ölabscheid

In [43]:
import re

def extract_numbered_sections(text):
    """
    Extract sections starting with numbers followed by dots and connect them to the following words
    until a non-letter/non-space character is found.

    Args:
        text (str): The full OCR text string.

    Returns:
        list of tuples: Each tuple is (number_prefix, connected_text)
    """
    pattern = re.compile(r"(\d+(?:\.\d+)*\.)\s")  # Match number-dot with a space after it
    
    results = []
    pos = 0
    length = len(text)
    
    while pos < length:
        match = pattern.search(text, pos)
        if not match:
            break
        
        number_prefix = match.group(1)  # e.g. '1.' or '1.2.'
        start = match.end()  # position after the matched number and space
        
        # Collect all words following the number prefix until hitting a non-letter, non-space char
        connected_words = []
        i = start
        while i < length:
            c = text[i]
            if c.isalpha() or c.isspace():
                connected_words.append(c)
                i += 1
            else:
                break
        
        connected_text = "".join(connected_words).strip()
        results.append((number_prefix, connected_text))
        
        pos = i  # continue search after the collected section
    
    return results

extract = extract_numbered_sections(text)
print(extract)

[('1.', 'Umgebungsbedingungen\nUmgebungstemperatur\nmax'), ('1.', 'UMGEBUNGSBEDINGUNGEN'), ('2.', 'PROJEKTIERTE\nBETRIEBSBEDINGUNGEN'), ('2.', 'Projektierte\nBetriebsbedingungen'), ('3.', 'HOCHSTZULÄSSIGE\nSättigungstemperatur'), ('4.', 'EINSTELLWERTE FÜR\nVerdichtersaugstutzen\nSICHERHEITSVENTILE'), ('5.', 'EINSTELLWERTE DER\nAufladung des Verdichters\nnein\nnein\nSICHERHEITSGERÄTE GEGEN\nUNZULÄSSIGE DRÜCKE'), ('3.', 'Hochstzulässige Betriebsdrücke'), ('6.', 'EINSTELLWERTE DER\nSICHERHEITSGERÄTE GEGEN\nKältemittelkreislauf'), ('7.', 'SONSTIGE EINSTELLWERTE VON\nSCHALTERN UND ARMATUREN'), ('4.', 'Einstellwerte für Sicherheitsventile'), ('8.', 'RICHTWERTE BETRIEB DES\nSicherheitsventil'), ('9.', 'HAUPTBAUTEILE\nzur'), ('10.', 'SENSOREN')]


In [42]:
process_text = separate_on_number(text)
print(process_text)

['Technische Angaben', 'Grass6', 'KAB', 'Schraubenverdichteraggregat', 'Actriocreticn Eqciamcii', 'FH-SGC', 'Übersicht', '1.', 'Umgebungsbedingungen', 'Umgebungstemperatur', "max.40 'C", "min.5 'C", '1. UMGEBUNGSBEDINGUNGEN', '2', 'frei von Flüssigkeitsniederschlägen', '2. PROJEKTIERTE', 'BETRIEBSBEDINGUNGEN', '2', '2. Projektierte', 'Betriebsbedingungen', '3. HOCHSTZULÄSSIGE', 'Sättigungstemperatur', '~5 *C', "+5 'C", 'BETRIEBSDRÜCKE', 'Verdichtersaugdruck to', 'Überhitzung im', '10 K', '10 K', '4. EINSTELLWERTE FÜR', 'Verdichtersaugstutzen', 'SICHERHEITSVENTILE', '2', 'Sättigungstemperatur', "+27 'C", "30 'C", 'Verdichtungsenddruck tK', '5. EINSTELLWERTE DER', 'Aufladung des Verdichters', 'nein', 'nein', 'SICHERHEITSGERÄTE GEGEN', 'UNZULÄSSIGE DRÜCKE', '3.', 'Hochstzulässige Betriebsdrücke', '6. EINSTELLWERTE DER', 'SICHERHEITSGERÄTE GEGEN', 'Kältemittelkreislauf', '23 bar Überdruck', 'UNZULÄSSIGE TEMPERATUREN', '3', 'Ölkreislauf', '23 bar Überdruck', '7. SONSTIGE EINSTELLWERTE VON',

In [41]:
sorted_lines = sort_toc_lines(valid_lines)
for i in sorted_lines:
    print(i)

1. UMGEBUNGSBEDINGUNGEN
2. PROJEKTIERTE
2. Projektierte
3. HOCHSTZULÄSSIGE
4. EINSTELLWERTE FÜR
4. Einstellwerte für Sicherheitsventile
5. EINSTELLWERTE DER
6. EINSTELLWERTE DER
7. SONSTIGE EINSTELLWERTE VON
8. RICHTWERTE BETRIEB DES
9. HAUPTBAUTEILE
10. SENSOREN
